In [79]:
# Imports

import numpy as np
import pandas as pd
import math
from antropy.antropy import entropy
import scipy
import scipy.signal
from envelope_derivative_operator.energy_operators import edo
from scipy.signal import periodogram, welch
from math import factorial, log
from sklearn.preprocessing import StandardScaler

In [ ]:
# Install necessary git repositories and pip installations

!git clone https://github.com/ktapani/Neonatal_Seizure_Detection.git
!git clone https://github.com/raphaelvallat/antropy.git
!git clone https://github.com/otoolej/envelope_derivative_operator.git
!pip install numba


In [6]:
%%time
# Read your respective window lengths file

df = pd.read_csv('filtered_babydf1sec.csv')

Wall time: 49.8 s


In [7]:
df

,0,1,2,3,4,5,6,7,8,9,...,663,664,665,666,667,668,669,670,671,672
0,0.684977,0.000000,0.292855,0.231423,0.499586,0.499322,0.672551,0.589906,0.747296,0.730789,...,0.028259,0.000000,0.027857,0.026016,0.066603,0.066553,0.095165,0.099653,0.132543,0.0
1,0.603158,0.833715,0.988354,0.822274,0.964054,0.918113,1.000000,0.755126,0.859063,0.620478,...,0.847722,0.791029,0.765582,0.723788,0.684644,0.615294,0.569372,0.518365,0.492546,0.0
2,0.827522,0.845448,0.955505,0.964454,1.000000,0.958826,0.944877,0.858996,0.755199,0.603973,...,0.918920,0.970277,1.000000,0.986674,0.971188,0.919860,0.863231,0.791238,0.713989,0.0
3,0.553916,0.471768,0.496251,0.608217,0.242712,0.217160,0.050715,0.143590,0.492903,0.635133,...,1.000000,0.999075,0.978012,0.943948,0.898813,0.832318,0.771626,0.703417,0.635989,0.0
4,0.389567,0.110689,0.000000,0.088504,0.118008,0.373595,0.435299,0.630278,0.928438,0.878670,...,0.785817,0.869272,0.940712,0.978989,1.000000,0.978191,0.942568,0.884459,0.780479,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383266,0.351476,0.472400,0.560105,1.000000,0.953081,0.582986,0.454187,0.427109,0.643399,0.620735,...,0.447839,0.596891,0.790467,1.000000,0.935383,0.594624,0.322721,0.447295,0.470114,0.0
383267,0.367142,0.531578,0.520828,0.214738,0.142540,0.372213,0.471551,0.241573,0.472458,0.093821,...,0.582060,0.442349,0.304768,0.364624,0.737840,1.000000,0.991711,0.772035,0.422163,0.0
383268,0.401376,0.608964,0.767277,1.000000,0.947401,0.484306,0.479490,0.475581,0.562292,0.487600,...,0.711916,0.688094,0.481399,0.084449,0.000000,0.075309,0.255715,0.486890,0.538963,0.0
383269,0.495757,0.412482,0.522240,0.212557,0.000000,0.179796,0.424438,0.661746,0.822805,0.617764,...,0.885890,0.759935,0.626263,0.488283,0.420768,0.596742,0.746918,0.751903,0.639968,0.0


<ins>Features considered in this notebook:</ins>

Total number of features considered = 11

- Mean
- Standard Deviation
- Kurtosis
- Skewness
- Shannon Entropy
- Activity
- Mobility
- Complexity
- Permutation Entropy
- Sample Entropy
- Approximate Entropy

In [8]:
# Hyperparams

window_length = 32


In [15]:
# %%time
# class NeonatalSeizureFeatures:
#     def __init__(self, row):
#         self.row = row
        
#     def skewness(self):
#         row = np.array(self.row)
#         row = row[:-1]
#         row = np.reshape(row, (21, window_length))
#         return (pd.Series(scipy.stats.skew(x, axis = 0, bias = False) for x in row))
    
    
# df_new = df.apply(lambda row: NeonatalSeizureFeatures(row).skewness(), axis = 1)

# df_new

Wall time: 5min 9s


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,-0.684344,-0.538593,-0.050992,-0.147113,-0.332855,0.156239,-0.257704,0.458084,0.199772,-0.089432,...,-0.215208,0.032215,0.118505,0.336897,0.214986,0.155961,0.338291,-0.377303,-1.936153,-0.000160
1,-0.062917,-0.055817,0.444097,0.089039,0.064485,-0.108347,0.424448,0.283813,-0.115612,0.144330,...,-0.179585,-0.142535,-0.085676,-0.241981,-0.200108,0.236962,-0.023803,-0.473674,-0.668736,1.751572
2,-0.145929,0.141554,-0.366421,0.257935,-0.112991,0.394517,-0.459955,0.194811,-0.021682,-0.002517,...,-0.256683,0.610447,-0.428388,0.434873,-0.400321,0.431798,-0.055480,0.108150,-1.925841,-0.139829
3,-0.174870,-0.550887,-0.952013,-0.946418,-0.227243,-0.192704,-0.998450,-0.079464,-0.997243,-0.037701,...,-0.644297,-0.158065,-0.574579,-0.034452,-0.220210,0.092368,-0.014633,0.294755,-1.723526,0.186293
4,-0.626088,-0.126920,0.514374,-0.102467,-0.129304,-0.336979,0.480137,-0.215187,0.032631,0.012131,...,-0.074478,-0.036161,-0.168621,-0.023750,0.317586,0.185429,-0.043430,-0.688818,-1.312343,0.187890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47871,0.554647,0.405337,0.654949,0.573514,0.422593,0.694235,0.355365,0.576686,-0.096675,0.383224,...,0.492626,0.356185,0.708717,-0.917423,0.212259,0.289629,0.029835,0.363740,-0.537799,0.205204
47872,-0.454933,-0.229896,-1.939226,0.485113,-0.587486,0.337810,-0.350310,-0.030279,-0.157104,0.528821,...,2.040335,-0.207425,0.549105,-0.458528,-0.186809,0.062689,0.403778,-0.588687,-0.398810,-1.692393
47873,-0.460754,-0.509863,-0.294692,1.108320,0.039623,-0.074952,0.367238,0.517058,0.768060,0.747768,...,0.844578,-0.019208,0.026966,0.733151,0.037885,0.043114,0.180367,0.418598,-0.711069,-1.133341
47874,0.587074,-0.341405,0.349596,0.642027,0.476135,0.086225,-0.150980,-0.969072,-1.149471,-0.431528,...,0.760455,0.454315,0.380758,-0.117407,-0.549923,0.613369,0.202710,-0.270475,-0.708810,0.386415


### Helper Methods:

In [9]:
def hMob(x):
    row = np.array(x)
    return (np.sqrt(np.var(np.gradient(x)) / np.var(x)))

### Feature Methods:

In [10]:
# Feature Methods

def feature_mean(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(np.mean(x, axis = 0) for x in row))

def feature_stddev(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(np.std(x, axis = 0) for x in row))

def kurtosis(row):
    row = np.array(row)
    annotation = row[-1]
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(scipy.stats.kurtosis(x, axis = 0, bias = False) for x in row))

def skewness(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(scipy.stats.skew(x, axis = 0, bias = False) for x in row))
    
def spectral_entropy(row, sf = 32, nperseg = window_length, axis = 1):
    row = np.array(row)
    annotation = row[-1]
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    _, psd = welch(row, sf, nperseg=nperseg, axis=axis)
    psd_norm = psd / psd.sum(axis=axis, keepdims=True)
    se = - np.where(psd_norm == 0, 0, psd_norm * np.log(psd_norm) / np.log(2)).sum(axis=axis)
    return pd.Series(se)

def hjorthActivity(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(np.var(x, axis = 0) for x in row))

def hjorthMobility(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(np.sqrt(np.var(np.gradient(x)) / np.var(x)) for x in row))

def hjorthComplexity(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series((hMob(np.gradient(x)) / hMob(x)) for x in row))

def permutation_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(entropy.perm_entropy(x) for x in row))

def sample_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(entropy.sample_entropy(x) for x in row))

def approximate_entropy(row):
    row = np.array(row)
    row = row[:-1]
    row = np.reshape(row, (21, window_length))
    return (pd.Series(entropy.app_entropy(x) for x in row))

In [11]:
list_of_feature_methods = [feature_mean, feature_stddev, kurtosis, skewness, spectral_entropy, hjorthActivity, hjorthMobility, 
                          hjorthComplexity, permutation_entropy, sample_entropy, approximate_entropy]

In [12]:
%%time
df_list = list()
for i, j in zip(list_of_feature_methods, range(len(list_of_feature_methods))):
    print("Epoch %d ..." % (j+1))
    df_temp = df.apply(lambda row: i(row), axis = 1)
    df_list.append(df_temp)

new_df = pd.concat(df_list, axis = 1)
new_df

Epoch 1 ...
Epoch 2 ...
Epoch 3 ...
Epoch 4 ...
Epoch 5 ...
Epoch 6 ...
Epoch 7 ...
Epoch 8 ...
Epoch 9 ...
Epoch 10 ...
Epoch 11 ...
Wall time: 3h 20min 45s


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.705170,0.627579,0.563296,0.561497,0.661709,0.500225,0.714688,0.464326,0.471334,0.412236,...,0.280471,0.207685,0.146087,0.136807,0.331072,0.180035,0.099877,0.092088,0.454638,0.102746
1,0.573662,0.507454,0.542473,0.483116,0.511353,0.437124,0.533494,0.495543,0.399331,0.550337,...,0.190806,0.056649,0.101368,0.277073,0.190806,0.281063,0.233324,0.429936,0.213984,0.145052
2,0.636943,0.520640,0.488823,0.529611,0.541663,0.511655,0.665044,0.549172,0.471088,0.532921,...,0.323282,0.216965,0.101368,0.101368,0.241633,0.118247,0.181526,0.216965,0.348141,0.185484
3,0.484862,0.562332,0.611532,0.441587,0.489014,0.396626,0.460126,0.454295,0.441191,0.557765,...,0.213984,0.190806,0.190806,0.184780,0.168446,0.120074,0.281881,0.315683,0.485606,0.187477
4,0.538846,0.536673,0.568350,0.550591,0.624571,0.474852,0.563601,0.410797,0.557898,0.472718,...,0.227373,0.072037,0.198405,0.477990,0.232353,0.148396,0.180035,0.367465,0.367984,0.169919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383266,0.384066,0.493491,0.404637,0.596175,0.419632,0.378368,0.454108,0.283672,0.364143,0.408382,...,0.307222,0.116756,0.189315,0.271782,0.250914,0.149886,0.126036,0.101368,0.195859,0.136807
383267,0.440657,0.546737,0.481236,0.497694,0.578748,0.415171,0.500591,0.510396,0.543095,0.438918,...,0.181526,0.090597,0.241633,0.213984,0.257212,0.267793,0.191624,0.146087,0.205150,0.306403
383268,0.345382,0.472884,0.345414,0.476114,0.349591,0.397218,0.423759,0.405204,0.551230,0.402841,...,0.196542,0.362711,0.294278,0.090597,0.158494,0.146087,0.535182,0.101368,0.191650,0.191981
383269,0.503639,0.457108,0.522277,0.459348,0.463334,0.553888,0.471511,0.712468,0.583607,0.610103,...,0.341843,0.237834,0.190806,0.295633,0.383390,0.181526,0.298978,0.101368,0.220179,0.248359


In [13]:
feature_df = pd.concat([new_df, df[df.columns[-1]]], axis = 1)
feature_df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,672
0,0.705170,0.627579,0.563296,0.561497,0.661709,0.500225,0.714688,0.464326,0.471334,0.412236,...,0.207685,0.146087,0.136807,0.331072,0.180035,0.099877,0.092088,0.454638,0.102746,0.0
1,0.573662,0.507454,0.542473,0.483116,0.511353,0.437124,0.533494,0.495543,0.399331,0.550337,...,0.056649,0.101368,0.277073,0.190806,0.281063,0.233324,0.429936,0.213984,0.145052,0.0
2,0.636943,0.520640,0.488823,0.529611,0.541663,0.511655,0.665044,0.549172,0.471088,0.532921,...,0.216965,0.101368,0.101368,0.241633,0.118247,0.181526,0.216965,0.348141,0.185484,0.0
3,0.484862,0.562332,0.611532,0.441587,0.489014,0.396626,0.460126,0.454295,0.441191,0.557765,...,0.190806,0.190806,0.184780,0.168446,0.120074,0.281881,0.315683,0.485606,0.187477,0.0
4,0.538846,0.536673,0.568350,0.550591,0.624571,0.474852,0.563601,0.410797,0.557898,0.472718,...,0.072037,0.198405,0.477990,0.232353,0.148396,0.180035,0.367465,0.367984,0.169919,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383266,0.384066,0.493491,0.404637,0.596175,0.419632,0.378368,0.454108,0.283672,0.364143,0.408382,...,0.116756,0.189315,0.271782,0.250914,0.149886,0.126036,0.101368,0.195859,0.136807,0.0
383267,0.440657,0.546737,0.481236,0.497694,0.578748,0.415171,0.500591,0.510396,0.543095,0.438918,...,0.090597,0.241633,0.213984,0.257212,0.267793,0.191624,0.146087,0.205150,0.306403,0.0
383268,0.345382,0.472884,0.345414,0.476114,0.349591,0.397218,0.423759,0.405204,0.551230,0.402841,...,0.362711,0.294278,0.090597,0.158494,0.146087,0.535182,0.101368,0.191650,0.191981,0.0
383269,0.503639,0.457108,0.522277,0.459348,0.463334,0.553888,0.471511,0.712468,0.583607,0.610103,...,0.237834,0.190806,0.295633,0.383390,0.181526,0.298978,0.101368,0.220179,0.248359,0.0


In [14]:
feature_df.to_csv('Full_feature_data1sec.csv', index = False)

In [118]:
feature_df1.columns = [i for i in range(232)]
feature_df1.columns

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            222, 223, 224, 225, 226, 227, 228, 229, 230, 231],
           dtype='int64', length=232)

In [114]:
np.isinf(feature_df1).values.any()

False

In [112]:
feature_df1 = feature_df.replace([np.inf, -np.inf], np.nan)
feature_df1.dropna(inplace = True)

In [116]:
feature_df1.reset_index(drop = True, inplace = True)

## Principal Component Analysis (PCA):

In [15]:
# Imports

from sklearn.decomposition import PCA

In [16]:
# Set hyperparams for PCA

n_components = 20
random_state = 32

### PCA 20

In [119]:
pca_20 = PCA(n_components = n_components, random_state = random_state)
feature_df_20 = pca_20.fit_transform(feature_df1[feature_df1.columns[:-1]])
# feature_df_20 = pca_20.fit_transform(feature_df1[feature_df1[[-1]]])

In [120]:
pca_20_df = pd.DataFrame(data = feature_df_20)
pca_20_df = pd.concat([pca_20_df, feature_df1[feature_df1.columns[-1]]], axis = 1)
pca_20_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,231
0,4.901421,2.701643,-1.275170,5.143429,-1.620967,0.613150,1.286259,-1.417769,1.444722,0.600224,...,0.060839,1.288825,-1.794129,0.458089,-1.073227,0.326684,-1.218720,-0.406938,-2.917282,0.0
1,0.863469,-5.155784,19.391457,3.268594,4.865219,0.716133,1.389465,-2.069688,-0.711067,-0.462562,...,-1.037438,-0.175020,-0.961911,0.866116,0.116357,-0.843982,-0.674588,-0.588865,0.109306,0.0
2,-6.782143,2.158658,2.189458,0.071391,1.237314,0.105731,0.060489,-0.475559,-0.271702,-0.060264,...,-0.058390,0.020884,-0.124925,0.076326,0.022127,-0.099033,-0.052237,-0.094348,0.041938,0.0
3,-7.053188,2.439245,1.538124,-0.132499,-0.224322,-0.010770,0.068195,-0.152156,-0.053641,-0.049488,...,-0.070432,-0.007473,-0.035539,-0.013059,-0.008530,-0.138042,-0.035067,-0.041577,-0.008063,0.0
4,-7.535043,3.178896,0.499794,-0.285159,-0.245198,-0.032130,0.100292,-0.220618,-0.253111,-0.046353,...,-0.073012,-0.005609,-0.119076,0.154522,0.022232,-0.011616,-0.018695,-0.026029,0.018659,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16291,2.121631,0.114495,-3.814118,1.318359,0.716929,-0.409302,3.169799,0.978609,0.101912,1.310513,...,0.066802,0.060389,1.356242,4.030198,-2.779573,0.908256,0.892026,1.440066,1.452993,0.0
16292,7.051724,0.046287,-3.450367,-5.499100,3.477877,0.270105,1.225670,-1.294833,-1.269587,-0.139015,...,-0.934864,0.667068,0.386159,0.555513,-1.243616,0.222214,0.239342,-0.141758,-0.135718,0.0
16293,-3.022786,-3.315476,-3.797521,-0.893076,0.438185,0.728091,-0.032998,-0.376513,1.373492,0.412405,...,0.973510,0.179913,-0.635228,1.117988,0.627298,-0.204403,0.180860,0.707815,0.536029,0.0
16294,5.998321,1.029297,-4.115128,-4.276564,-0.218342,-0.376820,2.220890,-1.687479,-0.395006,0.691122,...,-0.916872,0.898624,0.312184,2.311207,-0.437540,-0.726567,-0.122660,-1.774690,0.921198,0.0


In [121]:
pca_20_df.to_csv('1sec/PCA_20_features.csv', index = False)

### PCA 50

In [122]:
n_components = 50

pca_50 = PCA(n_components = n_components, random_state = random_state)
feature_df_50 = pca_50.fit_transform(feature_df1[feature_df1.columns[:-1]])

pca_50_df = pd.DataFrame(data = feature_df_50)
pca_50_df = pd.concat([pca_50_df, feature_df1[feature_df1.columns[-1]]], axis = 1)
pca_50_df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,231
0,4.901421,2.701643,-1.275170,5.143429,-1.620970,0.613131,1.286226,-1.417766,1.444427,0.600575,...,0.570364,-0.886126,-0.982307,-0.496790,-0.022562,-0.273687,-0.463349,-0.050930,0.801801,0.0
1,0.863469,-5.155784,19.391457,3.268593,4.865218,0.716138,1.389489,-2.069694,-0.710727,-0.461043,...,0.200579,-0.153199,-0.368352,-0.341452,-0.129752,-0.302006,-0.181581,0.478378,0.159127,0.0
2,-6.782143,2.158658,2.189458,0.071391,1.237313,0.105730,0.060494,-0.475554,-0.271635,-0.059872,...,0.153351,-0.044525,-0.039783,0.017995,-0.005723,0.014173,0.013893,-0.093317,-0.037416,0.0
3,-7.053188,2.439245,1.538124,-0.132499,-0.224322,-0.010769,0.068199,-0.152157,-0.053590,-0.049310,...,0.131352,-0.069443,-0.152969,0.032218,0.066243,0.084304,-0.032498,-0.099924,0.058492,0.0
4,-7.535043,3.178896,0.499794,-0.285159,-0.245198,-0.032130,0.100292,-0.220618,-0.253105,-0.046304,...,-0.002161,-0.006512,0.016940,0.006546,0.002581,-0.020468,0.001606,-0.005654,0.003170,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16291,2.121631,0.114495,-3.814118,1.318360,0.716924,-0.409301,3.169799,0.978602,0.101853,1.310425,...,-1.338026,-0.512642,0.503093,-0.327117,0.025187,-0.431047,0.222354,0.452380,-0.353413,0.0
16292,7.051724,0.046287,-3.450367,-5.499101,3.477877,0.270105,1.225675,-1.294856,-1.269620,-0.138700,...,-0.428840,0.173946,-0.462803,0.143407,-0.377900,-0.379543,0.331191,0.161700,-0.224093,0.0
16293,-3.022786,-3.315476,-3.797521,-0.893076,0.438187,0.728078,-0.033028,-0.376491,1.373422,0.413041,...,-0.180644,-0.363027,0.291971,-0.138437,-0.195820,-0.505695,0.029818,0.249897,-0.098363,0.0
16294,5.998321,1.029297,-4.115128,-4.276564,-0.218340,-0.376820,2.220898,-1.687488,-0.394971,0.691774,...,0.198149,0.404801,0.529178,0.758978,0.010270,-0.992147,0.239563,0.222264,-0.214098,0.0


In [123]:
pca_50_df.to_csv('1sec/PCA_50_features.csv', index = False)

### PCA 70

In [124]:
n_components = 70

pca_70 = PCA(n_components = n_components, random_state = random_state)
feature_df_70 = pca_70.fit_transform(feature_df1[feature_df1.columns[:-1]])

pca_70_df = pd.DataFrame(data = feature_df_70)
pca_70_df = pd.concat([pca_70_df, feature_df1[feature_df1.columns[-1]]], axis = 1)

pca_70_df.to_csv('1sec/PCA_70_features.csv', index = False)

### PCA 100

In [126]:
n_components = 100

pca_100 = PCA(n_components = n_components, random_state = random_state)
feature_df_100 = pca_100.fit_transform(feature_df1[feature_df1.columns[:-1]])

pca_100_df = pd.DataFrame(data = feature_df_100)
pca_100_df = pd.concat([pca_100_df, feature_df1[feature_df1.columns[-1]]], axis = 1)

pca_100_df.to_csv('1sec/PCA_100_features.csv', index = False)